<table align="left" style="border-style: hidden" class="table"> <tr> <td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon256.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Spring 2018</h4><p>Ani Adhikari</div></td></tr></table><!-- not in pdf -->

In [ ]:
!pip install pillow # Setup: run once

In [ ]:
import pickle
import numpy as np
from prob140 import *
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.style.use('fivethirtyeight')
import matplotlib.cm as cm
import matplotlib.colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from PIL import Image

In [ ]:
#run this code
def set_tile_to_color(image,tile,color):
    xmin,xmax,ymin,ymax = 0,0,0,0
    if tile//10 == 0:
        xmin = image.shape[0]-145
        xmax = image.shape[0]
        ymin = 145 + 93*((9-tile)%10)
        ymax = 145 + 93*((10-tile)%10) if tile !=0 else image.shape[1]
    elif tile//10 == 1:
        ymin = 0
        ymax = 145
        xmin = 145 + 93*((9-tile)%10)
        xmax = 145 + 93*((10-tile)%10)
        if tile == 10:
            xmin,xmax = image.shape[0]-145,image.shape[0]
    elif tile//10 ==2:
        xmin,xmax = 0,145
        ymin = 145 + 93*((tile-1)%10)
        ymax = ymin + 93
        if tile == 20:
            ymin,ymax = 0,145
    elif tile//10 == 3:
        ymin,ymax = image.shape[1]-145,image.shape[1]
        xmin = 145 + 93*((tile-1)%10)
        xmax = xmin + 93
        if tile == 30:
            xmin = 0
            xmax = 145
    mask = np.expand_dims(image[xmin:xmax,ymin:ymax,3]==0,2)
    mask = np.tile(mask,4)
    new_slot = np.tile(color,(xmax-xmin,ymax-ymin,1))*mask + image[xmin:xmax,ymin:ymax]*(1-mask)
    image[xmin:xmax,ymin:ymax] = new_slot

def plot_board(intensities,lighten=1):
    if len(intensities) == 42:
        intensities[30] += intensities[40] + intensities[41]
        intensities = intensities[:40]
    sort = sorted(intensities)
    norm = matplotlib.colors.Normalize(vmin=sort[0], vmax=lighten*sort[-2], clip=True)
    cmap = cm.Blues
    arr = np.array(Image.open('Lab_07_data/monopoly.png').convert('RGBA'))[20:-20,20:-20]
    for i, c in enumerate(intensities):
        set_tile_to_color(arr,i,255*np.array(cmap(norm(c))))        
    plt.figure(figsize=(15,15),dpi=200)
    plt.imshow(arr)
    plt.axis('off')
    
    

In [ ]:
#run this code
color_maps = {'Blue': 'blue',
 'Green': 'green',
 'Light Blue': 'cyan',
 'nan': 'white',
 'Orange': 'orange',
 'Pink': 'pink',
 'Purple': 'purple',
 'Railroad': 'black',
 'Red': 'red',
 'Utilities': 'gray',
 'Yellow': 'yellow'}

def profit_over_time(info_table):
    grouped = info_table.group('Colors',collect=sum).sort(4,descending=True)
    plt.figure(figsize=(8,8))
    roi_index = info_table.column_index("Expected Return Per Turn")
    cost_index = info_table.column_index("Cost")
    
    for row in grouped.rows:
        turns = np.arange(60)
        plt.plot(turns,row[roi_index]*turns - row[cost_index],label=row[0],c=color_maps[row[0]],linestyle=('--' if row[0] == 'Railroad' else '-'))
    plt.legend();
    plt.title('Average Profit vs Number of Turns');

# Lab 7: Board Game Night

In this lab, we'll apply Markov Chain theory and the basic probabilistic tools we've covered in this class to  analyze strategies and phenomena in two popular board games. 

Because a proper in-depth analysis of a board game can get involved, you are required to focus only on one game (the other game will be optional). We recommend that you read through both games to see which one you'd prefer doing. Remember to run all the cells above this one before starting.


*Fill in this cell with which game you've chosen to analyze (i.e which one we'll grade)*


# [Snakes and Ladders](#Game1)
# [Monopoly](#Game-2)

# Game 1

## Snakes and Ladders


[Snakes and Ladders](https://en.wikipedia.org/wiki/Snakes_and_Ladders) (or alternatively called *Chutes and Ladders*) is a board game where two or more players snake through tiles on a board, where *ladders* can signficantly promote the position of pieces, and chutes can demote position. The game is one of pure luck, and no player skill is required, making it useful for young audiences and probabilistic analysis. Our analysis will be using the board below, but can easily be adapted to boards of different sizes and with different snake and ladder positions. 


![Chutes and Ladders](Lab_07_data/snakes.png)



### General rules

- On each turn, each player rolls a dice (or spin a spinner), and move that number of steps. If the player lands on the start of a ladder or a snake, they move to the end of that passage.
- The game ends once the first person finishes the game by reaching the final tile.

In our analysis, we'll focus on two questions of interest

1. How long does a game take? How does this vary as the number of players increases?
2. How much of an advantage does the first person rolling get?

## Part 1: Modelling The Game

### Rules of the Game

- There are 100 positions on the board (0 - Start) 1, 2, ... (99 - Goal)
- On each turn, a fair dice is rolled, and the character moves forward by that many steps
- If the character lands on the start of a ladder or snake, then the character moves to the end of the passage
- If the character move would take the character past the goal, then the character doesn't move


### 1.1

Let's define $X_0, X_1, X_2 \dots$ to be the positions of a certain player every timestep. Is this a Markov Chain? If so, is it irreducible? Aperiodic?

*Your answer here*

In [ ]:
states = np.arange(100)

The function `ending_location(j)` takes a state $j$, and if a passage (snake or ladder) starts on that state, then returns the state corresponding to the end of the passage. If no passage starts on the state, then returns $j$. The output of the function corresponds to the state that the character ends at if they land on state $j$.

In [ ]:
passages = [(1, 18), (4, 12), (6, 2), (9, 26), (13, 32), (15, 72), (16, -1), (20, 35), (24, 8), (36, 57), (37, 3), (46, 88), (49, 4), (52, 34), (54, -1), (55, 74), (58, 21), (59, 76), (71, 84), (73, 26), (79, 96), (81, 43), (83, 57), (86, 89), (90, 23), (95, 85), (97, 61)]

def ending_location(j):
    for passage_start,passage_end in passages:
        if j == passage_start:
            return passage_end
    return j

### 1.2

Write code that generates the transition matrix for the markov chain in `snakes_ladders_matrix`. 

In [ ]:

snakes_ladders_matrix = np.zeros((100,100))

for start_state in range(100):
    for j in range(1,7): # Possible Dice Rolls
        # Remember to handle snakes and laddeers using `ending_location()`
        # Remember to handle when the move would take the user past the goal
        ...

### 1.3

Make the Markov Chain `snakes_and_ladders_mc` using the `MarkovChain.from_matrix(...)` function

In [ ]:

snakes_and_ladders_mc = ...

### 1.3

Use the `.simulate_path()` method to plot some example paths of a player on this Markov Chain for 100 turns. Remember that players start on tile 0 

In [ ]:

...

Can you identify the times when the player landed on a snake? A ladder? Do you recognize any absorbing states in the Markov Chain? Run the simulation multiple times, and try to identify game behaviours.

*Your analysis here*

## Part 2: Analyzing Game Lengths

### 2.1 (** On Paper **)

Let's investigate the distribution of the time taken for a single player to reach the goal state: this will serve as an important stepping stone in our analysis. 

Denote the time it takes to reach the goal state by a random variable $T$. Fill in the following equivalence between the markov chain at a certain time, $X_{t}$, and the CDF of $T$.

$$P(X_t~~\underline{\hspace{3cm}}) = P(T \leq t)$$


Notice that $T$ can take on an infinite number of values. We'll analytically approximate by considering $T < 600$. The number $600$ is arbitrary, chosen just large enough such that essentially all games terminate by then (in fact $99.9\%$ of games end by $600$ time steps). 


### 2.2

Use the equivalence you found in **2.1** to populate the array `prob_in_goal`, where `prob_in_goal[i] = `$P(T \leq i)$. 


In [ ]:

prob_in_goal = np.zeros(600)
for t in range(0,600): 
    ...

plt.plot(range(0,400),prob_in_goal[:400]) # We'll only look at the first 400
plt.title('$P(T \leq t)$');

### 2.3 (**On Paper**)

In Snakes and Ladders, $n$ players play until the first one finishes: denote each individual's finishing time to be $T_1,T_2, \dots T_n$ and the time the game is won to be $T^*$. 

Write $T^*$ as a function of the individual finishing times $T_1,T_2,\dots T_n$. 


### 2.4

Fill out the function `get_prob_finished`

Input:
`n` - the number of people playing

Output:
`prob_finished` - array corresponding to the CDF of $T^*$: i.e. `prob_finished[i] = `$P(T^* \leq i)$. 

In [ ]:

def get_prob_finished(n):
    prob_finished = ...
    return prob_finished

### 2.5

Fill out the function `distribution_from_cdf(cdf_probabilities)` which takes in an array of CDF probabilities $P(T \leq t)$, and outputs an array with probabilities $P(T = t)$

Use it with the `get_prob_finished` function to find the distributions when only $1$ person is playing, and when $4$ players are playing.

In [ ]:

def distribution_from_cdf(cdf_probabilities):
    ...

time_with_1 = Table().values(np.arange(600)).probability(distribution_from_cdf(get_prob_finished(1)))
time_with_4 = Table().values(np.arange(600)).probability(distribution_from_cdf(get_prob_finished(4)))

### 2.5

Use the `Plots` function to plot the distribution of the number of turns a game lasts for $1$ and $4$ players respectively. Also, list out the expected amount of time for each possible $n$

In [ ]:

print("Expected Time (1 Person): ", ...)
print("Expected Time (4 Person): ", ...)

... # Plot the distributions

## Part 3: Fairness

Since snakes and ladders is a game of sheer luck, the game is completely out of the hands of the player as soon as it starts. However, the first player to play has a slight advantage, since he/she will always roll first. In this section, we'll quantify this difference by computing $P(\text{Player 1 wins})$, and comparing it to the probability if the game had been completely fair.


### 3.1 (**On Paper**)


Once again, let's define $T_1,T_2,\dots T_n$ to be the amount of time it takes for each player to get to the goal. Express the quantity $P(\text{Player 1 wins})$ as an infinite sum which uses the distribution function $P(T=t)$ and the tail probabilities $P(T \geq t)$.

### 3.2

Fill out the function `p_first_person_win`, which computes $P(\text{Player 1 wins})$. We'll approximate the infinite sum in the previous part by considering the first 600 terms.

In [ ]:

def p_first_person_win(n):
    tail_probabilities = np.concatenate([[1],1-prob_in_goal])[:-1] # Array containing P(T >= k)
    distribution_probabilities = time_with_1.column(1) # Array containing P(T = k)

    return ...

One way we can assess the fairness is to see how much more likely are you to win, as compared to a completely fair game, where your chance of winning is $$P(\text{Win}) = \frac{1}{n}$$ 

The cell below computes the relative advantages for a variety of $n$, where the relative advantage is given by

$$\text{Relative Advantage} = \dfrac{P(\text{First person wins})}{\frac{1}{n}}$$

In [ ]:
def relative_advantage(n):
    return p_first_person_win(n) / (1/n)

ns = np.arange(1,10)
advantages = [relative_advantage(n) for n in ns]
plt.plot(ns,advantages)

### 3.3

Make some observations in the final cell about how the relative advantage changes as the number of players in the game increases.

*Your Insights Here*

# Submission

In [ ]:
import gsExport
gsExport.generateSubmission("Lab_07.ipynb")

# Game 2

## Monopoly

Monopoly is a board game, where players move around a board and purchase and develop properties, with the goal of bankrupting all the other players. Monopoly is an immensely complicated game, since players must decide what properties to buy, what deals to make with other players, and where to develop houses and hotels. Despite this, the movement in the game involves no user skill, and so one can analyze patterns in where players land to evaluate what the most lucrative properties are, and the best strategies to win the game.



<img src="http://koaning.io/theme/images/monopoly.jpg" alt="Drawing" style="width: 600px;"/>



In this analysis, we'll focus on the following three questions:

1. What tiles are landed on most often?
2. What are the most valuable properties to own in early-game?
3. What are the most valuable properties to own in late-game?

The rules for Monopoly are notoriously complicated, and so we make some additional assumptions to more easily model movement along the board (these assumptions can be lifted, and is a great side-project to investigate).

### Movement Rules (Simplified)

1. Each turn, the two dice are rolled, and the player moves that many spaces (no rolling twice if you got doubles)
2. Only way to go to jail is to land on 'Go to Jail' or draw an appropriate Chance/Community Chest card. (i.e. not 3 doubles)
3. Chance/CC Cards are sampled with replacement. We'll only consider the cards that affect board position
4. The only way to leave jail is to roll doubles, or wait 3 turns.

### Representing as a Markov Chain

Let's consider $X_0,X_1,\dots $ to be the position of a player in Monopoly. With a few tweaks, we can model $X_0,X_1 \dots$ as a Markov Chain. 


The naive model is for each state in the Markov Chain to correspond to a board tile. However, under this model, the the chain would not satisfy the Markov property since one can only be in Jail for three turns, and the state space doesn't capture how long the user has already been at jail. To avoid this issue, we'll have three states, each corresponding to a different time in Jail. More precisely, the state space will be

$$\mathcal{S} = \text{Board tiles} + \{ \text{Jail Turn 2}, \text{Jail Turn 3} \}$$

## Part 1: Coding the Monopoly Markov Chain

Run the following cell, which defines the `board_names` and `states` arrays.

`board_names` - a list with the names of the tiles in order

`states` - a list corresponding to the states in our Markov Chain, which includes each tile and two additional states for Jail.

In [ ]:
board_names = [
    'GO','Mediterranean','Com1','Baltic','Income','Reading RR','Oriental','Chance1','Vermont','Connecticut',
    'Visiting Jail','St Charles','Electric','States','Virginia','Penn RR','St James','Com2','Tennessee','New York',
    'Free Parking','Kentucky','Chance2','Indiana','Illinois','BO RR', 'Atlantic', 'Ventnor', 'Water', 'Marvin Gardens',
    'Jail','Pacific','North Carolina','Com3','Pennsylvania','Short Line','Chance3','Park Place','Luxury Tax','Boardwalk',
]

states = board_names + ['Jail Turn 2','Jail Turn 3']

The Markov chain corresponding to player position on the Monopoly board involves a more complicated transition function than we've previously worked with in class. There are three main types of moves happening in Monopoly:

- Moves from a **normal tile** (just rolling the dice)
- Moves from **Jail** (you can only escape Jail by rolling doubles, or waiting 3 turns)
- Moves from **community chest** or **chance** tile (cards can teleport you to different parts of the board)

In order to keep our code organized and understandable, we'll write the function `transition_function` which assumes the existence of the following functions

- `normal_transition_probability` - Handles transition probabilities starting at a normal tile
- `jail_transition_probability` - Handles moves starting from Jail tiles
- `chance_transition_probability` - Handles moves starting from Chance Tiles
- `com_chest_transition_probability` - Handles moves starting from Community Chest tiles

In [ ]:
def transition_function(s1, s2):
    
    if s2 == 'Jail Turn 2' and s1 != 'Jail': # Can only get to Jail Turn 2 from Jail
        return 0
    if s2 == 'Jail Turn 3' and s1 != 'Jail Turn 2': # Can only get to Jail Turn 3 from Jail Turn 2
        return 0
    
    # From Jail
    if s1 in ['Jail','Jail Turn 2', 'Jail Turn 3']:
        return jail_transition_probability(s1, s2)
    
    # Chance Cards
    if s1 in make_array('Chance1', 'Chance2', 'Chance3'):
        return chance_transition_probability(s1, s2)
    
    # Community Chest
    if s1 in make_array('Com1', 'Com2', 'Com3'):
        return com_chest_transition_probability(s1, s2)
    
    # Normal Cards
    return normal_transition_probability(s1,s2)


### 1.1 Normal Moves

Fill in the function `prob_of_dice` which will help you handle the transition probabilities when starting at a normal tile. The function takes a number `distance`, and returns the probability that two dice sum to `distance`. 

Be sure to handle edge cases, like when the distance is less than 2 or larger than 12.

In [ ]:

def prob_of_dice(distance):
    return ...


The function `normal_transition_probability` defined below returns the probability of transitioning from `tile1` to `tile2` if `tile1` was a normal tile. As you expect, this simply calls the function `prob_of_dice` you just wrote.

In [ ]:
def normal_transition_probability(s1,s2):
    index_1 = board_names.index(s1)
    index_2 = board_names.index(s2)
    distance = (index_2 - index_1) % 40
    return prob_of_dice(distance)

### 1.2 Jail

Fill in the blanks in the function `jail_transition_probability` which returns probabilities involving leaving Jail  states. 

Here are the general rules for making moves out of Jail squares:

- If you are on your 1st or 2nd turn in jail, you must roll doubles to leave. If you roll doubles, then you move that many tiles out of jail.
- If you are on your 3rd turn in jail, then the turn operates like a normal move

To make sure that the transitions are memoryless, we have a jail state for each possible time that you can be in jail 

- First turn: "Jail"
- Second turn: "Jail Turn 2"
- Third turn: "Jail Turn 3"

In [ ]:

def jail_transition_probability(s1, s2):
    assert s1 in ['Jail','Jail Turn 2', 'Jail Turn 3'], "jail_transition_probability not called from a Jail tile"

    # These are the tiles which are 2,4,6,8,10,12 away from jail
    doubles_away = make_array('Electric', 'Virginia','St James','Tennessee','Free Parking','Chance2') 
    
    # Jail: only leave if you roll doubles
    if s1 == 'Jail':
        
        if s2 in doubles_away:    
            return ...
        elif s2 == ...:
            return ...
        else:
            return ...
    
    if s1 == 'Jail Turn 2':
        if s2 in doubles_away:
            return ...
        elif s2 == ...:
            return ...
        return ...
    
    if s1 == 'Jail Turn 3':
        index_1 = board_names.index('Visiting Jail') # We're going to move as though we were at "Visiting Jail"
        index_2 = board_names.index(s2)
        distance = (index_2 - index_1) % 40
        return ...
    
    return 0

### 1.3 Chance and Community Chest Cards

#### Chance cards:

We'll implement chance cards as taking effect at the beginnning of the next turns. There are 17 chance cards, where $9$ cards change the position of the user, and $8$ providing either monetary gains or losses. For our Markov chain, we'll only consider cards that affect position. The $9$ cards are listed here:

* Advance to Go
* Advance to Illinois Ave.
* Advance to St. Charles Place
* Advance token to nearest Utility.
* Advance token to the nearest Railroad
* Go Back 3 Spaces
* Go to Jail
* Take a trip to Reading Railroad
* Take a walk on the Boardwalk

We've written the `chance_transition_probability` function, which handles transition probabilities for starting at Chance cards.

In [ ]:
def chance_transition_probability(s1,s2):
    index_1 = board_names.index(s1)
    index_2 = board_names.index(s2)
    distance = (index_2 - index_1) % 40
    # Multiple ways to end up at a location, so we have to sum up the probabilities.
    assert 'Chance' in s1, "chance_transition_probability not called starting at a Chance Card"
    
    
    p = 0
    
    if s2 in make_array('GO', 'Illinois', 'St Charles', 'Jail', 'Reading RR', 'Boardwalk'):
        p += 1 / 17
    if index_2 == index_1 - 3: # Go back 3 spaces.
        p += 1 / 17
    
    # Next utility and railroad.
    if s1 == 'Chance1' and s2 in make_array('Electric', 'Penn RR'):
        p += 1 / 17
    if s1 == 'Chance2' and s2 in make_array('Water', 'BO RR'):
        p += 1 / 17
    if s1 == 'Chance3' and s2 in make_array('Electric', 'Reading RR'):
        p += 1 / 17        
    
    # Chance of getting there on dice roll
    p +=  (8 / 17) * prob_of_dice(distance)
    return p


#### Community Chest cards

There are 17 community chest cards, where exactly 2 cards affect board position.

* Advance to Go
* Go to Jail

Fill out the function `com_chest_transition_probability(s1,s2)`, which handles transition probabilities when starting at a community chest card. If you're stuck, get inspiration by looking at `chance_transition_probability`. 

In [ ]:

def com_chest_transition_probability(s1, s2):
    assert 'Com' in s1, "com_chest_transition_probability not called starting at a Community Chest Tile"

    index_1 = board_names.index(s1)
    index_2 = board_names.index(s2)
    distance = (index_2 - index_1) % 40
    
    p = 0
    if s2 in make_array('GO', 'Jail'):
        ...
        
    # Chance of getting there on dice roll
    p += ... * prob_of_dice(distance) # What should the modifier be here?
    return p

### 1.4 Putting it All Together


Initialize `monopoly_mc` to the Markov Chain corresponding to a player's location in a Monopoly game.

In [ ]:

monopoly_mc = ...

In [ ]:
# Debug Cell
# If you're having issues with probabilities not summing to 1, run this cell

for state,value in zip(monopoly_mc.states,monopoly_mc.matrix.sum(axis=1)):
    if not np.isclose(value,1):
        print("From %s: probability %f"%(state,value))


## Part 2: Where in Monopoly?

In this section, we'll look at how a player travels around the monopoly board, and the long-run proportion of time that a player spends in each state. This will form the basis for our cost analysis.

### 2.1

Plot a 30-turn path for a player using the  `.simulate_path` (remember that the player starts at GO). What do you notice about the path? Did the player go to Jail in the path that you plotted? How many times did your player cross GO? Did your player land on a Chance or Community Chest tile that moved the player to a completely different position?

In [ ]:
...


*Your comments here*

### 2.2

Let's analyze the steady state distribution of the player's position on the board. Save the steady state distribution to a variable called `steady_state_dist` and the array of transition probabilities into a variable called `steady_state`. Make some comments about the steady state distribution? Do the states with the highest probability make sense? What about the lowest? We've included the monopoly board image below for comparison.

In [ ]:
steady_state_dist = ...
steady_state = ...
steady_state_dist.sort(1,descending=True).show() # Show states sorted by steady state probabilities

*Your Comments Here*

<img src="http://koaning.io/theme/images/monopoly.jpg" alt="Drawing" style="width: 600px;"/>


### 2.3

We've created a function called `plot_board` which visualizes values for each tile on the Monopoly board. Run the following cell below, to see a visualization of the steady state.

In [ ]:
plot_board(steady_state,lighten=1.2)
plt.title('Most probable locations on board');

## Part 3: Property Valuations

Now that we've set up the Markov Chain, and looked at the steady state distribution of where the player is likely to be after a sufficient number of turns, let's use this to analyze what the best properties are to own and invest in during different stages of the game.

### Rules for Buying Property

1. Initial cost is buying the property: if one owns all the properties of a certain color, he/she can build houses on each property for a fixed sum, and after 4 houses, a hotel.
2. The owner of the property gets more money for a player landing on the property based on the number of houses (or hotel) on the property. Rent prices increase as you move along the board.
2. The rent for landing on a railroad is proportional to the number of railroads the owner possesses: for this analysis, we assume you own all 4 railroads - landing on a railroad pays out \$200
3. Utility rents are a function of the number of utilities owned and the number rolled on the dice. Assuming you own both, one can compute that the expected rent is \$70.

We'll define the **early-game** to be the time period when the user controls all the properties of a color, but **has no houses**

We'll define the **late-game** to be the end of the game, when the user has **hotels** on each property.

We've stored property values and costs in a file called `monopoly_info.csv`.

In [ ]:
monopoly_information = Table.read_table('Lab_07_data/monopoly_info.csv')
monopoly_information

### 3.1

Just to get a better hang of property costs and rents, let's plot the costs of each property, and the rent that each generates without any houses. 

In [ ]:
plot_board(monopoly_information['Property Cost'])
plt.title('Cost of each Property');

In [ ]:
plot_board(monopoly_information['No House Rent'])
plt.title('Rent of each Property');

### 3.2: Early Game Analysis

In the early game, all the properties have been bought, but no houses or hotels have been invested.

Let's assume you're playing against a single opponent. If your opponent's position has reached a steady state distribution, then the expected return per turn for a property $s$ is given by

$$ \text{Expected Return} = R(s)\pi(s)$$

where $R(s)$ is the rent generated from landing on the tile. In $t$ timesteps, we expect to make

$$ \text{Expected Return in t timesteps} = tR(s)\pi(s) $$

### 3.2

The rent generated in the early game is `monopoly_information['No House Rent']`.Fill out the code for the expected return per turn, and the profit in 60 turns. Remember that 

$$ \text{Profit} = \text{Revenue} - \text{Cost}$$


In [ ]:
early_game = monopoly_information.copy()
early_game['Expected Return Per Turn'] = ...
early_game['Cost'] = early_game['Property Cost']
early_game['Profit in 60 Turns'] = ...
early_game

How do we fare overall for each color group in monopoly? We'll split the tiles into 

- Group the properties by color
- Group the railroads together
- Group the Utilities together

In [ ]:
early_game_results = early_game.group('Colors',collect=sum)
early_game_results.sort('Profit in 60 Turns sum',descending=True)

The `profit_over_time` function charts the profit as a function of the number of turns played after buying.

In [ ]:
profit_over_time(early_game)

Finally, let's look at profit for each individual property

In [ ]:
plot_board(early_game['Profit in 60 Turns'],lighten=1.5) # How much reward after 60 turns
plt.title('Profit After 60 Turns (Early Game)');

### 3.3

Make some observations based on the above three graphs about what properties are most valuable, what color groups profit the best, and which seem to be better investments in the early game.

*Your Observations Here*

### 3.4: Late Game Analysis

Let's now consider the end-game, when all properties have hotels on them? What's the best set of investments to bankrupt your opponents?


In the late game, the rent generated is `monopoly_information['Hotel Rent']`. Just as in the early game, fill out the code for the expected return per turn, and the profit in 60 turns. 

In [ ]:

late_game = monopoly_information.copy()
late_game['Expected Return Per Turn'] = ...
late_game['Cost'] = late_game['Property Cost'] + 5*late_game['House Cost']
late_game['Profit in 60 Turns'] = ...
late_game

In [ ]:
late_game_results = late_game.group('Colors',collect=sum)
late_game_results.sort('Profit in 60 Turns sum',descending=True)

In [ ]:
profit_over_time(late_game)

In [ ]:
plot_board(late_game['Profit in 60 Turns'],lighten=1.5) # How much reward after 60 turns
plt.title('Profit After 60 Turns (Late Game)');

### 3.5

Make some observations based on the above three graphs about what properties are most valuable, what color groups profit the best, and which seem to be better investments in the late game. How have these groups changed from the early game?

*Your observations here*

## Part 4: Finale


In a short paragraph, describe what strategies in Monopoly you'd adopt as a result of this analysis. Did the results confirm any heuristic strategies in Monopoly that you previously developed? Do the numbers match up with games you've played in real life? What monopoly techniques does this analysis **not show** that you'd like to tell us about?

*Your response here*

# Submission

In [ ]:
import gsExport
gsExport.generateSubmission("Lab_07.ipynb")